`251202_banknote_auth.ipynb`에 작성하세요. # 137

# 지폐 위조 판별 성능 향상 비교 실험  

**Dataset: Banknote Authentication (Kaggle)**  
https://www.kaggle.com/datasets/ritesaluja/bank-note-authentication-uci-data

[우리반 노션 - Datasets Wiki](https://www.notion.so/shqkel/2bd9cb46e5e280208276dffaada6cfba?source=copy_link)

<img src="https://d.pr/i/FoSZqa+"/>

---

- [ ] 1. 데이터 로드 및 확인
  - CSV 파일을 로드하고 컬럼명(`variance`, `skewness`, `curtosis`, `entropy`, `label`)을 지정하라.  
  - 데이터의 기본 통계값과 라벨 분포를 확인하라.
    - 데이터 shape  
    - label 분포  
    - 컬럼 의미 요약


- [ ] 2. 데이터 분할 및 스케일링
  - 데이터를 train/test = 8:2로 분할하라.  
  - `StandardScaler`로 스케일링하라.  
  - 스케일링 전/후 값을 비교하라.
  

- [ ] 3. Logistic Regression 모델 학습 및 성능 평가
  - LogisticRegression 모델을 학습시키고 성능평가 출력하라.


- [ ] 4. SVM(RBF Kernel) 모델 학습 및 성능 평가
  - RBF SVM을 학습하고 정확도를 출력하라.  
  - Logistic Regression과 성능 차이를 비교하라.
  - 두 모델의 차이가 발생하는 이유를 설명하라.

- [ ] 5. 두 모델 성능 비교표 작성
  - 아래 형식을 따라 성능평가 표를 작성하라.

| 모델 | 정확도 | 장점 | 단점 |
|------|--------|--------|--------|
| Logistic Regression | ? | ? | ? |
| SVM (RBF) | ? | ? | ? |

- 모델별 성능 비교 그래프 (bar chart)를 시각화 하라
